<a href="https://colab.research.google.com/github/profteachkids/subtitle_generator/blob/main/subtitle_generate_color_animation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Generate Text

In [28]:
!wget -nc https://github.com/profteachkids/subtitle_generator/raw/main/Dosis-Bold.ttf
!wget -nc https://raw.githubusercontent.com/profteachkids/subtitle_generator/main/subtitle_timeline_templates.py

File ‘Dosis-Bold.ttf’ already there; not retrieving.

File ‘subtitle_timeline_templates.py’ already there; not retrieving.



In [29]:
import os
import numpy as np
import re
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageOps
import requests
from io import BytesIO
import shutil
from datetime import date
from subtitle_timeline_templates import timeline_start,timeline_end,clipitem

colors={'w':(255, 255, 255, 255),
    'k':(0, 0, 0, 255),   #black
    'r':(255, 0, 0, 255),    #red
    '1':(255, 0, 221, 255),  #pink
    'b':(0,0,255,255), #blue
    '2':(0,255,69,255),
    '3':(128,0,128,255), #purple
    'o':(255,165,0,255), #orange
    'y':(255,255,0,255)} #yellow

text="""this is a test
of how well this
Minecraft subtitle works
server IP in Discord""" 

rootname='subtitles'+date.today().strftime('%d%b%y')

root='./'+rootname
try:
    shutil.rmtree(rootname)
except:
    pass
os.makedirs(root)

imw,imh=1920,250

fnt = ImageFont.truetype("Dosis-Bold.ttf", 115)

In [30]:
text_lines=text.split('\n')
for i,text_line in enumerate(text_lines):
    z = re.match(r"%\w{3}%", text_line)
    if not z:
        text_lines[i]='%wkk%'+text_line
    
i=0
w_start=0
filenames=[]
w_starts, w_ends=[],[]
for text_line in text_lines:
    z = re.split(r"(%\w{3}%)", text_line)
    color_strings = z[1::2]
    text_strings = z[2::2]

    full_text_line = ''.join(text_strings)
    w_end = w_start + len(full_text_line.split())
    w_starts.append(w_start)
    w_ends.append(w_end)

    filename = f'a_{i:04d}_{w_start:04d}_{w_end:04d}.png'
    fullname=root+ '/' + filename
    i+=1
    w_start = w_end

    im = Image.new('RGBA', (imw,imh), (0,0,0,0))
    d = ImageDraw.Draw(im)
    imface = Image.new('RGBA', (imw,imh), (0,0,0,0))
    d2 = ImageDraw.Draw(imface)

    w,h=d.textsize(full_text_line,fnt)
    x,y = (imw-w)//2, (imh-h)//2
    prior_text=''
    for color_string, text_string in zip(color_strings,text_strings):
        font_c = color_string[1]
        outline_c = color_string[2]
        shadow_c = color_string[3]

        d.text((x,y), text_string, font=fnt, fill=colors[shadow_c])
        d2.text((x,y), text_string, font=fnt, fill=colors[font_c], stroke_width=7, stroke_fill=colors[outline_c])

        prior_text+=text_string
        x+=d.textsize(text_string,fnt)[0]

    im = im.filter(ImageFilter.GaussianBlur(radius=30))
    imnp_raw=np.asarray(im)
    imnp=imnp_raw.copy()
    imnp[:,:,3]=(imnp_raw[:,:,3]*(255.0/imnp_raw[:,:,3].max())).astype(np.uint8)
    imfacenp = np.asarray(imface)
    blank = np.zeros((imh,imw,4),dtype=np.uint8)
    compnp=np.where((imnp[:,:,3]>0)[:,:,None], imnp, blank)
    compnp=np.where((imfacenp[:,:,3]>200)[:,:,None], imfacenp, compnp)

    Image.fromarray(compnp,'RGBA').save(fullname)
    filenames.append(filename)
shutil.make_archive(rootname,'zip',root_dir =root)

'/home/carlos/github/content/subtitles08Oct21.zip'

##Imports and Silerio

In [31]:
!pip install biopython
!pip install bidict
!pip install torchaudio
!wget -nc https://github.com/profteachkids/subtitle_generator/raw/main/en_v5.jit

File ‘en_v5.jit’ already there; not retrieving.



In [32]:
import subprocess as sp
import os
import torch
import pandas as pd
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from bidict import bidict
import string
from itertools import groupby

class Decoder():
    def __init__(self,labels):
        self.labels = labels
        self.blank_idx = self.labels.index('_')
        self.space_idx = self.labels.index(' ')

    def process(self,
                probs, wav_len, word_align):
        assert len(self.labels) == probs.shape[1]
        for_string = []
        argm = torch.argmax(probs, axis=1)
        align_list = [[]]
        for j, i in enumerate(argm):
            if i == self.labels.index('2'):
                try:
                    prev = for_string[-1]
                    for_string.append('$')
                    for_string.append(prev)
                    align_list[-1].append(j)
                    continue
                except:
                    for_string.append(' ')
                    warnings.warn('Token "2" detected a the beginning of sentence, omitting')
                    align_list.append([])
                    continue
            if i != self.blank_idx:
                for_string.append(self.labels[i])
                if i == self.space_idx:
                    align_list.append([])
                else:
                    align_list[-1].append(j)

        string = ''.join([x[0] for x in groupby(for_string)]).replace('$', '').strip()

        align_list = list(filter(lambda x: x, align_list))

        if align_list and wav_len and word_align:
            align_dicts = []
            linear_align_coeff = wav_len / len(argm)
            to_move = min(align_list[0][0], 1.5)
            for i, align_word in enumerate(align_list):
                if len(align_word) == 1:
                    align_word.append(align_word[0])
                align_word[0] = align_word[0] - to_move
                if i == (len(align_list) - 1):
                    to_move = min(1.5, len(argm) - i)
                    align_word[-1] = align_word[-1] + to_move
                else:
                    to_move = min(1.5, (align_list[i+1][0] - align_word[-1]) / 2)
                    align_word[-1] = align_word[-1] + to_move

            for word, timing in zip(string.split(), align_list):
                align_dicts.append({'word': word,
                                    'start_ts': round(timing[0] * linear_align_coeff, 2),
                                    'end_ts': round(timing[-1] * linear_align_coeff, 2)})

            return string, align_dicts
        return string

    def __call__(self,
                 probs: torch.Tensor,
                 wav_len: float = 0,
                 word_align: bool = False):
        return self.process(probs, wav_len, word_align)

torch.set_grad_enabled(False)

model = torch.jit.load("en_v5.jit", map_location=torch.device('cpu'))
model.eval()
decoder=Decoder(model.labels)

DEVNULL = open(os.devnull, "w")



def ffmpeg_load_audio(
    filename,
    sr=44100,
    mono=False,
    normalize=True,
    in_type=np.int16,
    out_type=np.float32,
):
    channels = 1 if mono else 2
    format_strings = {
        np.float64: "f64le",
        np.float32: "f32le",
        np.int16: "s16le",
        np.int32: "s32le",
        np.uint32: "u32le",
    }
    format_string = format_strings[in_type]
    command = [
        "ffmpeg",
        "-i",
        filename,
        "-f",
        format_string,
        "-acodec",
        "pcm_" + format_string,
        "-ar",
        str(sr),
        "-ac",
        str(channels),
        "-",
    ]
    p = sp.Popen(command, stdout=sp.PIPE, stderr=DEVNULL, bufsize=4096)
    bytes_per_sample = np.dtype(in_type).itemsize
    frame_size = bytes_per_sample * channels
    chunk_size = frame_size * sr  # read in 1-second chunks
    raw = b""
    with p.stdout as stdout:
        while True:
            data = stdout.read(chunk_size)
            if data:
                raw += data
            else:
                break
    audio = np.frombuffer(raw, dtype=in_type).astype(out_type)
    if channels > 1:
        audio = audio.reshape((-1, channels)).transpose()
    if audio.size == 0:
        return audio, sr
    if issubclass(out_type, np.floating):
        if normalize:
            peak = np.abs(audio).max()
            if peak > 0:
                audio /= peak
        elif issubclass(in_type, np.integer):
            audio /= np.iinfo(in_type).max
    return audio, sr

## Timeline

In [40]:

def align(rawtext, voice_words, start_times, end_times):

    all_words = set(rawtext).union(set(voice_words))

    word_dict = bidict(zip(all_words, range(len(all_words))))
    word_dict["-"] = "-"
    voice = [word_dict[word] for word in voice_words]
    script = [word_dict[word] for word in rawtext]
    align = pairwise2.align.globalxx(voice, script, gap_char =['-'], one_alignment_only=True)[0]
    print(align)
    matches=[]
    mismatches=[]
    mismatch_voice=word_dict.inverse[align.seqA[0]]   
    mismatch_text=word_dict.inverse[align.seqB[0]]
    v_end_pos=0
    v_start_pos=0
    t_end_pos=0
    t_start_pos=0
    for i,(v, t) in enumerate(zip(align.seqA[1:], align.seqB[1:])):
        # print(word_dict.inverse[v], word_dict.inverse[t])
        if v==t:
            mismatches.append([t_start_pos, t_end_pos, start_times[v_start_pos], end_times[v_end_pos], mismatch_voice, mismatch_text])
            v_end_pos+=1
            v_start_pos=v_end_pos
            t_end_pos+=1
            t_start_pos=t_end_pos
            mismatch_voice=word_dict.inverse[v]
            mismatch_text=word_dict.inverse[t]
        else:
            if word_dict.inverse[v] != '-' :
                v_end_pos+=1
            if word_dict.inverse[t] != '-' :
                t_end_pos+=1
            mismatch_voice+=' ' + word_dict.inverse[v]
            mismatch_text+=' ' + word_dict.inverse[t]


    mismatches.append([t_start_pos, t_end_pos, start_times[v_start_pos], end_times[v_end_pos], mismatch_voice, mismatch_text])

    idx=[]
    times=[]
    for t_start_pos, t_end_pos, start_time, end_time, v, t in mismatches:
        print('-'*30)
        print(v)
        print(t)
        print(t_start_pos,t_end_pos, start_time, end_time)
        d={}

        t2 = t.translate(t.maketrans(string.punctuation, " " * len(string.punctuation)))
        t2 = " ".join(t2.split())

        idx.append(t_start_pos)
        times.append(start_time)

    idx.append(t_end_pos+1)
    times.append(end_time)

    return np.array(idx),np.array(times)


def create_timeline(duration, idx, times, silent_sections):

    timeline=timeline_start(duration)
    for filename,w_start,w_end in zip(filenames,w_starts,w_ends):
        start, end = (np.interp([w_start, w_end], idx, times)*60).astype(np.int64)
        # test=np.logical_and(end>silent_sections[:,0], end<silent_sections[:,1])
        # if np.sum(test)>0:
        #     end=silent_sections[np.nonzero(test)[0],0][0]
        #     print(filename,end)

        
        timeline+=clipitem(path,filename,start,end)
    timeline+=timeline_end
    print(timeline)
    with open('Timeline'+date.today().strftime('%d%b%y')+'.xml','w') as f:
        f.write(timeline)
    

In [41]:
def to_text(f):
    audio, sr = ffmpeg_load_audio(
        f, sr=16000, in_type=np.float32, out_type=np.float32
    )
    audio = audio.astype(np.float32)[0]

    res = []
    batch_size =  3*sr #Silerio batch size

    # split audio by silent sections
    N_silence_sampling=1600
    N = audio.size // N_silence_sampling
    resampled = np.mean(np.abs(audio[:N_silence_sampling*N].reshape(N,N_silence_sampling)),axis=1)
    silence=np.where(resampled<0.02)[0]
    edges=np.where(silence[1:]-silence[:-1]>1)[0]
    silent_sections=np.c_[np.take(silence,np.insert(edges+1,0,0)[:-1]),np.take(silence,edges)]
    silent_sections = silent_sections[silent_sections[:,1]-silent_sections[:,0]>15]*N_silence_sampling

    if len(silent_sections)>0:
        audio_sections = [audio[x[0]:x[1]] for x in zip(silent_sections[:-1,1],silent_sections[1:,0])]
        audio_sections.append(audio[silent_sections[-1,-1]:])
        audio_sections_start=silent_sections[:,1]
        if silent_sections[0,0]!=0:
            audio_sections_start=np.insert(audio_sections_start,0,0)
            audio_sections=[audio[:silent_sections[0,0]]] + audio_sections
    else:
        audio_sections=[audio]
        audio_sections_start=[0]


    for audio_section, start in zip(audio_sections, audio_sections_start):

        batch_start=start
        N_batch = audio_section.size // batch_size
        N_batch = 1 if N_batch<1 else N_batch
        audio_section = np.expand_dims(audio_section,axis=0)
        # Split audio into smaller batches 
        batches = np.array_split(audio_section, N_batch, axis=1)
        for i, batch in enumerate(batches):
            input=torch.from_numpy(batch)
            output = model(input)[0]
            decoded=decoder(output.cpu(), batch.size, word_align=True)
            if len(decoded)==2:
                s,dlist = decoded
                for d in dlist:
                    d["start_ts"] = (d["start_ts"] + batch_start)/sr
                    d["end_ts"] = (d["end_ts"] +  batch_start)/sr
                    print(d["word"], d["start_ts"], d["end_ts"])
                    res.append(d)
                batch_start+=batch.size

    df = pd.DataFrame(res)
    return df['word'].values, df['start_ts'].values, df['end_ts'].values, (silent_sections/sr*60).astype(np.int64)

In [42]:

path='D:/Downloads/'+rootname
rawtext = re.sub(r"%\w{3}%",'',text)
rawtext = re.sub(r"[,.!?-] ", " ", rawtext, 0, re.MULTILINE)
rawtext = rawtext.lower()
rawtext = rawtext.split()
voice_words, start_times, end_times, silent_sections = to_text("audio.wav")
idx, times = align(rawtext, voice_words, start_times, end_times)
duration = int(times[-1]*60)
create_timeline(duration, idx, times, silent_sections)

this 0.0397725 0.278409375
is 0.278409375 0.4772725
a 0.4772725 0.67613625
test 0.67613625 0.994318125
of 0.994318125 1.193181875
how 1.193181875 1.392045625
well 1.4715906250000002 1.7102275
this 1.7897725 2.028409375
minecraft 2.028409375 2.82386375
sub 2.9034093750000003 3.142045625
bricks 3.69886375 4.17613625
server 4.57386375 4.9318181249999995
ip 4.9318181249999995 5.1306818750000005
in 5.3693181249999995 5.5681818750000005
thiscord 5.5681818750000005 6.0852275
Alignment(seqA=[0, 4, 9, 8, 6, 15, 12, 0, 14, 13, 1, '-', '-', 16, 7, 5, 10, '-'], seqB=[0, 4, 9, 8, 6, 15, 12, 0, 14, '-', '-', 3, 11, 16, 7, 5, '-', 2], score=12.0, start=0, end=18)
------------------------------
this
this
0 0 0.0397725 0.278409375
------------------------------
is
is
1 1 0.278409375 0.4772725
------------------------------
a
a
2 2 0.4772725 0.67613625
------------------------------
test
test
3 3 0.67613625 0.994318125
------------------------------
of
of
4 4 0.994318125 1.193181875
--------------------